In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-pdvpqwop/gym-walk_a92f47d81f254b08bb2eb79383d369bb
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-pdvpqwop/gym-walk_a92f47d81f254b08bb2eb79383d369bb
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=d033d286ced0d974870ce7ff5e37e404b0668d4e73f2ab648c02e47539a7f5ca
  Stored in directory: /tmp/pip-ephem-wheel-cache-lftawei3/wheels/60/02/77/2dd9f31df8d13bc7c014725f4002e29d0fc3ced5e8ac08e1cf
Successfully built gym-walk


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123);


In [ ]:

def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:


def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")


In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:

env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
goal_state = 15
LEFT, DOWN, RIGHT, UP = range(4)

In [ ]:
P

{0: {0: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  2: [(0.3333333333333333, 4, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)],
  3: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 1: {0: [(0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True)],
  1: [(0.3333333333333333, 0, 0.0, False),
   (0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False)],
  2: [(0.3333333333333333, 5, 0.0, True),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False)],
  3: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.3333333333333333, 0, 0.0, False)]},
 2:

In [ ]:

init_state

0

In [ ]:


state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:0 - reward:0.0 - done:False - info:{'prob': 0.3333333333333333}


In [ ]:

pi_frozenlake = lambda s: {
    0: RIGHT,
    1: DOWN,
    2: RIGHT,
    3: LEFT,
    4: DOWN,
    5: LEFT,
    6: RIGHT,
    7:LEFT,
    8: UP,
    9: DOWN,
    10:LEFT,
    11:DOWN,
    12:RIGHT,
    13:RIGHT,
    14:DOWN,
    15:LEFT #Stop
}[s]
print_policy(pi_frozenlake, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)

Policy:
| 00      > | 01      v | 02      > | 03      < |
| 04      v |           | 06      > |           |
| 08      ^ | 09      v | 10      < |           |
|           | 13      > | 14      v |           |


In [ ]:

print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_frozenlake, goal_state=goal_state) * 100,
    mean_return(env, pi_frozenlake)))

Reaches goal 10.00%. Obtains an average undiscounted return of 0.1000.


In [ ]:
pi_2 = lambda s: {
    0: DOWN,
    1: RIGHT,
    2: RIGHT,
    3: DOWN,
    4: LEFT,
    5: RIGHT,
    6: DOWN,
    7: LEFT,
    8: DOWN,
    9: RIGHT,
    10: LEFT,
    11: DOWN,
    12: RIGHT,
    13: DOWN,
    14: RIGHT,
    15: LEFT
}[s]


In [ ]:
# Name : Mohammed Saajid
# Register Number : 212223240093
print_policy(pi_2, P, action_symbols=('<', 'v', '>', '^'), n_cols=4)


Name: Archana k
Register Number: 212222240011
Policy:
| 00      v | 01      > | 02      > | 03      v |
| 04      < |           | 06      v |           |
| 08      v | 09      > | 10      < |           |
|           | 13      v | 14      > |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state) * 100,
    mean_return(env, pi_2)))

Reaches goal 6.00%. Obtains an average undiscounted return of 0.0600.


In [ ]:
success_pi2 = probability_success(env, pi_2, goal_state=goal_state) * 100
mean_return_pi2 = mean_return(env, pi_2)

print("\nYour Policy Results:")
print(f"Reaches goal: {success_pi2:.2f}%")
print(f"Average undiscounted return: {mean_return_pi2:.4f}")


Your Policy Results:
Reaches goal: 6.00%
Average undiscounted return: 0.0600


In [ ]:
success_pi1 = probability_success(env, pi_frozenlake, goal_state=goal_state) * 100
mean_return_pi1 = mean_return(env, pi_frozenlake)

print("\nComparison of Policies:")
print(f"First Policy - Success Rate: {success_pi1:.2f}%, Mean Return: {mean_return_pi1:.4f}")
print(f"Your Policy  - Success Rate: {success_pi2:.2f}%, Mean Return: {mean_return_pi2:.4f}")

if success_pi1 > success_pi2:
    print("\nThe first policy is better based on success rate.")
elif success_pi2 > success_pi1:
    print("\nYour policy is better based on success rate.")
else:
    print("\nBoth policies have the same success rate.")

if mean_return_pi1 > mean_return_pi2:
    print("The first policy is better based on mean return.")
elif mean_return_pi2 > mean_return_pi1:
    print("Your policy is better based on mean return.")
else:
    print("Both policies have the same mean return.")


Comparison of Policies:
First Policy - Success Rate: 10.00%, Mean Return: 0.1000
Your Policy  - Success Rate: 6.00%, Mean Return: 0.0600

The first policy is better based on success rate.
The first policy is better based on mean return.


In [ ]:
def policy_evaluation(pi, P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)

    while True:
        delta = 0
        for s in range(len(P)):
            v = 0
            a = pi(s)
            for prob, next_state, reward, done in P[s][a]:
                v += prob * (reward + gamma * V[next_state])
            delta = max(delta, abs(V[s] - v))
            V[s] = v

        if delta < theta:
            break

    return V


In [ ]:


V1 = policy_evaluation(pi_frozenlake, P,gamma=0.99)
print_state_value_function(V1, P, n_cols=4, prec=5)

State-value function:
| 00 0.11448 | 01 0.08191 | 02 0.13372 | 03 0.06586 |
| 04 0.15053 |           | 06 0.20562 |           |
| 08 0.30562 | 09 0.46997 | 10 0.48938 |           |
|           | 13 0.62915 | 14 0.80739 |           |


In [ ]:

V2 = policy_evaluation(pi_2, P, gamma=0.99)

print("\nState-value function for Your Policy:")
print_state_value_function(V2, P, n_cols=4, prec=5)



State-value function for Your Policy:
State-value function:
| 00 0.05568 | 01 0.03401 | 02 0.06904 | 03 0.03401 |
| 04 0.07905 |           | 06 0.10617 |           |
| 08 0.10481 | 09 0.21279 | 10 0.32173 |           |
|           | 13 0.32309 | 14 0.65598 |           |


In [ ]:

if np.sum(V1 >= V2) == len(V1):
    print("\nThe first policy is the better policy.")
elif np.sum(V2 >= V1) == len(V2):
    print("\nYour policy is the better policy.")
else:
    print("\nBoth policies have their merits.")



The first policy is the better policy.


In [ ]:
V1>=V2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [ ]:

if(np.sum(V1>=V2)==11):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)==11):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

Both policies have their merits.
